In [46]:
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
import matplotlib
import copy
import cv2
# matplotlib.font_manager._rebuild()
# plt.rcParams['font.family'] = 'Times New Roman' #全体のフォントを設定
%matplotlib inline

In [2]:
import sys
sys.path.append("/notebooks/nb_IEEE/opmap/")

In [3]:
import opmap
from opmap.rawCam import RawCam
from opmap.vmemMap import VmemMap
from opmap.apdMap import APDMap
from opmap.phaseMapHilbert import PhaseMapHilbert as PhaseMap
from opmap.phaseVarianceMap import PhaseVarianceMap

In [4]:
def FFT(vmem_1d, N, dt, fc_low=None, fc_high=None):
    fq = np.linspace(0, 1.0/dt, N)
    F = np.fft.fft(vmem_1d)
    if fc_low is not None:
        F[(fq <= fc_low)] = 0
    if fc_high is not None:
        F[(fq >= fc_high)] = 0
    
    F_abs = np.abs(F)
    F_abs_amp = F_abs/N
#     F_abs_amp[0] = F_abs_amp[0]/2
    F_abs_amp[0] = 0
    
    F_ifft = np.fft.ifft(F)
    F_ifft_real = F_ifft.real * 2
    
    return F_ifft_real

In [5]:
def isochronal(pmap, threshold):
    contour = np.zeros_like(pmap.data)

    for i in range(contour.shape[1]):
        for j in range(contour.shape[2]):        
            if pmap.roi[i,j] == 1:
                counter = 0
                for time in range(contour.shape[0]):
                    if time == 0:
                        pass
                    elif ((pmap.data[time-1, i, j] < threshold) * (pmap.data[time, i, j] > threshold)) == 1:
                        counter = 0

                    contour[time, i, j] = counter
                    counter += 1
                
    return contour

## 旋回興奮に対する冷却

In [7]:
import cv2

### 20181218-1/C001H001S0026

In [6]:
parent_path = '/mnt/Jalife/ExperimentData/'

data_path = os.path.join(parent_path, '20181218-1/C001H001S0026/')
print(data_path)
start = 29500
end = 30500
date = data_path.split('/')[-3]
session_name = data_path.split('/')[-2]
save_parent = os.path.join('/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/', '%s/%s/%s-%s' % (date, session_name, start, end))

raw = RawCam(path=data_path, cam_type='sa4',
          image_width=256, image_height=256, frame_start=start, frame_end=end)

vmem = VmemMap(raw)

#roi_png = cv2.imread('/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/20181121-1/C001H001S0007/roi.png', cv2.IMREAD_GRAYSCALE)
#roi_png = roi_png/255.0

#vmem.roi = roi_png
#vmem.data = vmem.data * vmem.roi

for i in range(vmem.data.shape[1]):
    for j in range(vmem.data.shape[2]):
        vmem.data[:, i, j] = FFT(vmem.data[:, i, j], vmem.data.shape[0], 0.002, fc_low=2, fc_high=50)

vmem.smooth(size=9)

/mnt/Jalife/ExperimentData/20181218-1/C001H001S0026/


In [42]:
os.makedirs(os.path.join(save_parent, 'pmap'))

In [7]:
pmap = PhaseMap(vmem)

In [47]:
for i in range(1000):
    dat = copy.deepcopy(pmap.data[i]) - 2
    dat = dat +(dat < -np.pi)*2*np.pi
    
    plt.imsave(os.path.join(save_parent, 'pmap/%s.png' % '{0:04d}'.format(i)),
               dat*cv2.resize(vmem.roi, (128, 128))*(-1), vmin=-np.pi, vmax=np.pi, cmap='jet')
    plt.close()

## 20181218-1/C001H001S0028

In [48]:
parent_path = '/mnt/Jalife/ExperimentData/'

data_path = os.path.join(parent_path, '20181218-1/C001H001S0028/')
print(data_path)
start = 21000
end = 22000
date = data_path.split('/')[-3]
session_name = data_path.split('/')[-2]
save_parent = os.path.join('/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/', '%s/%s/%s-%s' % (date, session_name, start, end))

raw = RawCam(path=data_path, cam_type='sa4',
          image_width=256, image_height=256, frame_start=start, frame_end=end)

vmem = VmemMap(raw)

#roi_png = cv2.imread('/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/20181121-1/C001H001S0007/roi.png', cv2.IMREAD_GRAYSCALE)
#roi_png = roi_png/255.0

#vmem.roi = roi_png
#vmem.data = vmem.data * vmem.roi

for i in range(vmem.data.shape[1]):
    for j in range(vmem.data.shape[2]):
        vmem.data[:, i, j] = FFT(vmem.data[:, i, j], vmem.data.shape[0], 0.002, fc_low=2, fc_high=50)

vmem.smooth(size=9)

/mnt/Jalife/ExperimentData/20181218-1/C001H001S0028/


In [49]:
os.makedirs(os.path.join(save_parent, 'pmap'))
pmap = PhaseMap(vmem)

In [50]:
for i in range(1000):
    dat = copy.deepcopy(pmap.data[i]) - 2
    dat = dat +(dat < -np.pi)*2*np.pi
    
    plt.imsave(os.path.join(save_parent, 'pmap/%s.png' % '{0:04d}'.format(i)),
               dat*cv2.resize(vmem.roi, (128, 128))*(-1), vmin=-np.pi, vmax=np.pi, cmap='jet')
    plt.close()

## batch phase variance analysis

In [51]:
analysis_path = ["/mnt/Jalife/ExperimentData/20181121-1/C001H001S0008/",
                 "/mnt/Jalife/ExperimentData/20181121-1/C001H001S0014/",
                 "/mnt/Jalife/ExperimentData/20181121-1/C001H001S0017/",
                 "/mnt/Jalife/ExperimentData/20181121-1/C001H001S0021/",
                 "/mnt/Jalife/ExperimentData/20181218-1/C001H001S0007/",
                 "/mnt/Jalife/ExperimentData/20181218-1/C001H001S0008/",
                 "/mnt/Jalife/ExperimentData/20181218-1/C001H001S0009/",
                 "/mnt/Jalife/ExperimentData/20181218-1/C001H001S0011/",
                 "/mnt/Jalife/ExperimentData/20181218-1/C001H001S0017/",
                 "/mnt/Jalife/ExperimentData/20181218-1/C001H001S0018/",
                 "/mnt/Jalife/ExperimentData/20181218-1/C001H001S0020/",
                 "/mnt/Jalife/ExperimentData/20181218-1/C001H001S0022/",
                 "/mnt/Jalife/ExperimentData/20181218-1/C001H001S0024/",
                 "/mnt/Jalife/ExperimentData/20181218-1/C001H001S0026/",
                 "/mnt/Jalife/ExperimentData/20181218-1/C001H001S0028/",
                 "/mnt/Jalife/ExperimentData/20181220-1/C001H001S0007/",
                 "/mnt/Jalife/ExperimentData/20181220-1/C001H001S0009/",
                 "/mnt/Jalife/ExperimentData/20181220-1/C001H001S0010/",
                 "/mnt/Jalife/ExperimentData/20181220-1/C001H001S0012/",
                 "/mnt/Jalife/ExperimentData/20181220-1/C001H001S0014/",
                 "/mnt/Jalife/ExperimentData/20181220-1/C001H001S0015/",
                 "/mnt/Jalife/ExperimentData/20181220-1/C001H001S0018/",
                 "/mnt/Jalife/ExperimentData/20181220-1/C001H001S0020/",
                 "/mnt/Jalife/ExperimentData/20181220-1/C001H001S0022/"
                ]

In [62]:
for analysis in analysis_path:
    #print(analysis)
    
    start = 0
    end = 10
    
    date = analysis.split('/')[-3]
    session_name = analysis.split('/')[-2]
    print(date, session_name)
    
    save_name = os.path.join('/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/raw', '%s_%s.png' % (date, session_name))
    
    raw = RawCam(path=analysis, cam_type='sa4',
          image_width=256, image_height=256, frame_start=start, frame_end=end)
    
    plt.imsave(save_name, raw.data[5]/np.max(raw.data[5]), vmin=0, vmax=1, cmap='gray')
    plt.close()

20181121-1 C001H001S0008
20181121-1 C001H001S0014
20181121-1 C001H001S0017
20181121-1 C001H001S0021
20181218-1 C001H001S0007
20181218-1 C001H001S0008
20181218-1 C001H001S0009
20181218-1 C001H001S0011
20181218-1 C001H001S0017
20181218-1 C001H001S0018
20181218-1 C001H001S0020
20181218-1 C001H001S0022
20181218-1 C001H001S0024
20181218-1 C001H001S0026
20181218-1 C001H001S0028
20181220-1 C001H001S0007
20181220-1 C001H001S0009
20181220-1 C001H001S0010
20181220-1 C001H001S0012
20181220-1 C001H001S0014
20181220-1 C001H001S0015
20181220-1 C001H001S0018
20181220-1 C001H001S0020
20181220-1 C001H001S0022


In [63]:
analysis_path = [["/mnt/Jalife/ExperimentData/20181121-1/C001H001S0008/", 29000, 30000,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181121-1_C001H001S0014.png"],
                 ["/mnt/Jalife/ExperimentData/20181121-1/C001H001S0014/", 29000, 30000,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181121-1_C001H001S0014.png"],
                 ["/mnt/Jalife/ExperimentData/20181121-1/C001H001S0017/", 21500, 22500,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181121-1_C001H001S0017.png"],
                 ["/mnt/Jalife/ExperimentData/20181121-1/C001H001S0021/", 29000, 30000,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181121-1_C001H001S0017.png"],
                 ["/mnt/Jalife/ExperimentData/20181218-1/C001H001S0007/", 29000, 30000,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181218-1_C001H001S0007.png"],
                 ["/mnt/Jalife/ExperimentData/20181218-1/C001H001S0008/", 29000, 30000,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181218-1_C001H001S0007.png"],
                 ["/mnt/Jalife/ExperimentData/20181218-1/C001H001S0009/", 29000, 30000,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181218-1_C001H001S0009.png"],
                 ["/mnt/Jalife/ExperimentData/20181218-1/C001H001S0011/", 29000, 30000,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181218-1_C001H001S0009.png"],
                 ["/mnt/Jalife/ExperimentData/20181218-1/C001H001S0017/", 29000, 30000,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181218-1_C001H001S0009.png"],
                 ["/mnt/Jalife/ExperimentData/20181218-1/C001H001S0018/", 24200, 25200,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181218-1_C001H001S0018.png"],
                 ["/mnt/Jalife/ExperimentData/20181218-1/C001H001S0020/", 6700, 7700,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181218-1_C001H001S0018.png"],
                 ["/mnt/Jalife/ExperimentData/20181218-1/C001H001S0022/", 25700, 26700,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181218-1_C001H001S0018.png"],
                 ["/mnt/Jalife/ExperimentData/20181218-1/C001H001S0024/", 29000, 30000,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181218-1_C001H001S0018.png"],
                 ["/mnt/Jalife/ExperimentData/20181218-1/C001H001S0026/", 29500, 30500,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181218-1_C001H001S0018.png"],
                 ["/mnt/Jalife/ExperimentData/20181218-1/C001H001S0028/", 21000, 22000,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181218-1_C001H001S0018.png"],
                 ["/mnt/Jalife/ExperimentData/20181220-1/C001H001S0007/", 29000, 30000,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181220-1_C001H001S0007.png"],
                 ["/mnt/Jalife/ExperimentData/20181220-1/C001H001S0009/", 29000, 30000,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181220-1_C001H001S0007.png"],
                 ["/mnt/Jalife/ExperimentData/20181220-1/C001H001S0010/", 29000, 30000,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181220-1_C001H001S0012.png"],
                 ["/mnt/Jalife/ExperimentData/20181220-1/C001H001S0012/", 9500, 10500,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181220-1_C001H001S0012.png"],
                 ["/mnt/Jalife/ExperimentData/20181220-1/C001H001S0014/", 29000, 30000,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181220-1_C001H001S0012.png"],
                 ["/mnt/Jalife/ExperimentData/20181220-1/C001H001S0015/", 29000, 30000,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181220-1_C001H001S0020.png"],
                 ["/mnt/Jalife/ExperimentData/20181220-1/C001H001S0018/", 10000, 11000,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181220-1_C001H001S0020.png"],
                 ["/mnt/Jalife/ExperimentData/20181220-1/C001H001S0020/", 7300, 8300,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181220-1_C001H001S0020.png"],
                 ["/mnt/Jalife/ExperimentData/20181220-1/C001H001S0022/", 29000, 30000,
                  "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181220-1_C001H001S0020.png"]
                ]

In [89]:
for analysis in analysis_path:
    
    data_path = analysis[0]
    start = analysis[1]
    end = analysis[2]
    mask_dir = analysis[3]
    
    date = data_path.split('/')[-3]
    session_name = data_path.split('/')[-2]
    
    print(data_path, start, end)
    print(date, session_name)
    
    raw = RawCam(path=data_path, cam_type='sa4',
          image_width=256, image_height=256, frame_start=start, frame_end=end)
    
    vmem = VmemMap(raw)
    
    roi_png = cv2.imread(mask_dir, cv2.IMREAD_GRAYSCALE)
    roi_png = roi_png/255.0

    vmem.roi = roi_png
    vmem.data = vmem.data * vmem.roi
    
    for i in range(vmem.data.shape[1]):
        for j in range(vmem.data.shape[2]):
            vmem.data[:, i, j] = FFT(vmem.data[:, i, j], vmem.data.shape[0], 0.002, fc_low=2, fc_high=50)

    vmem.smooth(size=9)
    
    pmap = PhaseMap(vmem)
    pvmap = PhaseVarianceMap(pmap, size=9)
    
    PS = (pvmap.data > 0.8)*1

    plt.imsave(os.path.join("/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/PS_trajectory/", "%s_%s.png" % (date, session_name)),
              np.sum(PS, axis=0), cmap="viridis")
    plt.close()

/mnt/Jalife/ExperimentData/20181121-1/C001H001S0008/ 29000 30000
20181121-1 C001H001S0008
/mnt/Jalife/ExperimentData/20181121-1/C001H001S0014/ 29000 30000
20181121-1 C001H001S0014
/mnt/Jalife/ExperimentData/20181121-1/C001H001S0017/ 21500 22500
20181121-1 C001H001S0017
/mnt/Jalife/ExperimentData/20181121-1/C001H001S0021/ 29000 30000
20181121-1 C001H001S0021
/mnt/Jalife/ExperimentData/20181218-1/C001H001S0007/ 29000 30000
20181218-1 C001H001S0007
/mnt/Jalife/ExperimentData/20181218-1/C001H001S0008/ 29000 30000
20181218-1 C001H001S0008
/mnt/Jalife/ExperimentData/20181218-1/C001H001S0009/ 29000 30000
20181218-1 C001H001S0009
/mnt/Jalife/ExperimentData/20181218-1/C001H001S0011/ 29000 30000
20181218-1 C001H001S0011
/mnt/Jalife/ExperimentData/20181218-1/C001H001S0017/ 29000 30000
20181218-1 C001H001S0017
/mnt/Jalife/ExperimentData/20181218-1/C001H001S0018/ 24200 25200
20181218-1 C001H001S0018
/mnt/Jalife/ExperimentData/20181218-1/C001H001S0020/ 6700 7700
20181218-1 C001H001S0020
/mnt/Jalife/

In [90]:
analysis = ["/mnt/Jalife/ExperimentData/20181121-1/C001H001S0017/", 21500, 22500,
            "/mnt/Omer/Project/03.LinearRegionalCooling/AnalysisResult/Experiment/IEEE/mask/20181121-1_C001H001S0017.png"]